# Flowsheet optimization: NGL feed composition sensitivity
## ROK model: M5; CO$_2$ tax rate: USD 45/tonne CO$_2$e; Region: EF-1 to EF-12

In [1]:
# simulation case specifications
model_code = 5
case_name = 'EF-Basin'
c_tax_rate = 4.5e-2

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                define_models, \
                                define_arcs, \
                                set_unit_model_variables, \
                                initialize_flowsheet, \
                                set_scaling_factors,\
                                update_model_after_initialization,\
                                vapor_only_to_vapor_liquid_reformulate,\
                                update_model_for_optimization, \
                                unfix_DOFs_pre_optimization, \
                                fix_DOFs_post_optimization

## Create flowsheet

In [3]:
m = create_flowsheet(model_code)

In [4]:
M_catalyst = 1167.003367 # kg

## Define equipment and connections

In [5]:
# define unit models
define_models(m, catalyst_mass = M_catalyst)
#define connections
define_arcs(m)

## Define inlet compositions

In [6]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_composition_dict = {}

for col in inlet_df.columns:
    if col == case_name:
        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)

inlet_flow_rate = 481.3888889

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188}

## Define constraints and set-points for equipment

In [7]:
set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)

## Scale model components

In [8]:
if model_code == 2 or model_code == 3:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
elif model_code == 4 or model_code == 5:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
else:
    pass

## Read-in initialization data from .json file

In [9]:
init_file_name = "./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format('Bakken', model_code)
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.0547945499420166,
 'etime_read_dict': 0.1697535514831543,
 'etime_read_suffixes': 0.0059833526611328125}

## Add post-initialization constraints

In [10]:
update_model_after_initialization(m)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)

Liquid phase flow is zero. Translator block fs.T102 liquid phase composition equations are being modified...

Degenerate constraints removed and liquid phase composition values set to 1e-8

Translator block fs.T102 liquid phase composition equations already modified, no degenerate constraints remaining.



## Read-in flowsheet convergence data from .json file

In [11]:
## Read-in flowsheet convergence data from .json fileinit_file_name = "./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.07535386085510254,
 'etime_read_dict': 0.1739206314086914,
 'etime_read_suffixes': 0.0}

## Costing

In [12]:
from src.emissions_calculations import calc_lhv_values, calculate_stream_energies, calculate_emissions, create_ghg_objective, delete_region_specific_components
from src.costing_function import add_costing,calculate_costs_for_objective
from src.utility_minimization_1d import (
    min_utility,
    PinchDataClass,
    heat_ex_data,
    gen_curves,
    print_HX_results,
    generate_curves,
    heat_data,
    pinch_calc,
    return_data
)

### Equipment costing

In [13]:
add_costing(m)

### Heat integration

In [14]:
### Heat Exchangers and reactors
min_utility(
    m.fs, [m.fs.H101, m.fs.H103, m.fs.R101], [m.fs.H102, m.fs.H104, m.fs.H105, m.fs.H106, m.fs.R102], 10.0
)
m.fs.Qs.fix()

### Emissions calculation

In [15]:
calc_lhv_values(m,case_name,'./LHV.xlsx','NGL_compositions.csv','NGL_fraction.csv')
calculate_stream_energies(m)
calculate_emissions(m,case_name,'emissions_factor_by_region.csv')
create_ghg_objective(m)
calculate_costs_for_objective(m,c_tax_flag=True, c_tax_val = c_tax_rate)

## Read-in costing-initialized flowsheet data from .json file with EF-Basin NGL composition

In [16]:
init_file_name = "./initialization_files/CISTAR_solve_with_costing_{}_C_tax_{}_M{}_purge_{}.json.gz".format(case_name,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.05059671401977539,
 'etime_read_dict': 0.15973210334777832,
 'etime_read_suffixes': 0.016731977462768555}

## Flowsheet initialization with 8 DoFs for multiple feed compositions: EF-1 to EF-12

In [17]:
update_model_for_optimization(m)

Model updates:


Constraint added for recycle ratio <= 8.0.

fs.H103 temperature lower and upper bounds added.

fs.R102 isothermal operation constraint added.

fs.H106 temperature lower bound updated.

fs.F102 temperature lower bound updated.

Objective function added.



## Dataframes to record multiple zone output

In [18]:
import pandas as pd

In [19]:
cols = ['region','butene', 'pentene', 'hexene', 'heptene', 'octene', 'nonene']
outlet_conc_df = pd.DataFrame(columns=cols)

LHV_cols = ['region','LHV_kJ_mol','MW_fuel','LHV_kJ_s']
LHV_df = pd.DataFrame(columns=LHV_cols)

In [20]:
upstream_emissions_factor_df = pd.read_csv('emissions_factor_by_region.csv')

In [21]:
# case name for sequential initialization
case_name_previous = 'EF-Basin'
for col in inlet_df.columns:
    if col == 'Species':
        pass
    elif col in ['Bakken','EF-Basin']:
        pass
    else:
        print('\n Region = {} \n'.format(col))
        region = col
        case_name_current = col
        
        if col != 'EF-Basin':
            if col == 'EF-9': # special case
                init_file_name = "./initialization_files/CISTAR_optimal_solution_{}_C_tax_{}_M{}_purge_{}_sequential_solve.json.gz".format('EF-8',m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
                ms.from_json(m, fname=init_file_name)
            else:
                init_file_name = "./initialization_files/CISTAR_optimal_solution_{}_C_tax_{}_M{}_purge_{}_sequential_solve.json.gz".format(case_name,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
                ms.from_json(m, fname=init_file_name)
            for i,r in inlet_df.iterrows():
                m.fs.M101.feed.mole_frac_comp[0, r['Species']].unfix()
                if r[col] == 0.0:
                    m.fs.M101.feed.mole_frac_comp[0, r['Species']].fix(1e-6)
                else:
                    m.fs.M101.feed.mole_frac_comp[0, r['Species']].fix(round(r[col],4))

            delete_region_specific_components(m)
            calc_lhv_values(m,case_name_current,'./LHV.xlsx','NGL_compositions.csv','NGL_fraction.csv')
            m.fs.upstream_emission_factor = upstream_emissions_factor_df[case_name_current].iloc[0]
        print("Upstream EF = ",m.fs.upstream_emission_factor())
        
        # Unfix DOFs
        unfix_DOFs_pre_optimization(m)
        
        # special cases, need to perturb initial point to avoid being stuck around infeasible solutions
        if case_name_current in ['EF-1','EF-6','EF-7','EF-10','EF-11']:
            # adjust initial point
            m.fs.H103.outlet.temperature.fix(653.0)
            m.fs.H103.outlet.temperature.unfix()

        DOF_initial = degrees_of_freedom(m)
        print("The initial DOF is {0}".format(DOF_initial))
        
        # Solve optimization problem
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6,
                          'bound_push': 1e-8,
                          'max_iter': 500,
                         }
        solve_status = solver.solve(m, tee=True)

        # Fix DOFs
        fix_DOFs_post_optimization(m)
        
        ms.to_json(m, fname="./initialization_files/CISTAR_optimal_solution_{}_C_tax_{}_M{}_purge_{}_sequential_solve.json.gz".format(case_name_current,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))
        
        conc_list = []
        for species_col in cols:
            if species_col == 'region':
                conc_list.append(region)
            else:
                conc_list.append(m.fs.F102.liq_outlet.flow_mol_phase_comp[0.0, 'Liq', species_col]())

        outlet_conc_df.loc[len(outlet_conc_df.index)] = conc_list
        
        print("====================  Region = {}  ======================".format(region))
        print(outlet_conc_df)


 Region = EF-1 

Upstream EF =  6.74
The initial DOF is 8
Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
   

  13  1.1344336e+01 8.28e-05 9.22e-04  -5.7 1.08e+00  -4.5 1.00e+00 1.00e+00f  1
  14  1.1343815e+01 7.81e-06 8.13e-05  -5.7 1.42e-01  -5.0 1.00e+00 9.04e-01f  1
  15  1.1343815e+01 1.19e-07 5.24e-06  -7.0 1.06e-02  -5.4 1.00e+00 1.00e+00h  1
  16  1.1343815e+01 1.79e-07 3.91e-08  -7.0 3.17e-02  -5.9 1.00e+00 1.00e+00h  1

Number of Iterations....: 16

                                   (scaled)                 (unscaled)
Objective...............:   1.1343815431452471e+01    1.1343815431452471e+01
Dual infeasibility......:   3.9110205812351190e-08    3.9110205812351190e-08
Constraint violation....:   8.3819031715393066e-09    1.7881393432617188e-07
Complementarity.........:   9.0909092132660090e-08    9.0909092132660090e-08
Overall NLP error.......:   9.0909092132660090e-08    1.7881393432617188e-07


Number of objective function evaluations             = 30
Number of objective gradient evaluations             = 17
Number of equality constraint evaluations            = 30
Number of ine

   1  1.1232625e+01 7.55e+06 2.90e+06  -1.0 3.10e+06    -  3.29e-01 3.89e-02f  2
   2  1.1231420e+01 7.23e+06 5.25e+06  -1.0 2.97e+06    -  4.10e-01 4.21e-02h  1
   3  1.1231297e+01 7.19e+06 8.04e+06  -1.0 3.10e+06    -  7.18e-01 5.18e-03h  1
   4  1.1230527e+01 6.91e+06 8.62e+06  -1.0 3.21e+06    -  8.38e-01 3.99e-02h  4
   5  1.1231395e+01 6.60e+06 8.35e+06  -1.0 3.06e+06    -  6.16e-01 4.40e-02h  5
   6  1.1237343e+01 1.10e+02 6.25e+06  -1.0 2.89e+06    -  4.96e-01 1.00e+00H  1
   7  1.1365485e+01 4.62e+00 2.04e+04  -1.0 3.32e+03    -  4.53e-01 1.00e+00f  1
   8  1.1474682e+01 9.46e+00 4.80e+03  -1.0 7.18e+03    -  7.64e-01 1.00e+00f  1
   9  1.1763189e+01 1.04e+02 6.81e+02  -1.0 2.98e+04    -  8.58e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.2024874e+01 5.70e+02 8.55e+01  -1.0 1.80e+05    -  9.88e-01 1.00e+00f  1
  11  1.1218288e+01 1.68e+03 4.39e+05  -1.7 1.96e+05    -  9.89e-01 9.34e-01h  1
  12  1.1384678e+01 1.46e+03

Upstream EF =  8.3
The initial DOF is 8
Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collect

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.1960018e+01 4.57e+02 1.78e+05  -1.0 2.69e+06    -  9.90e-01 4.44e-01f  1
  11  1.1986937e+01 2.48e+02 2.41e+04  -1.0 1.03e+06    -  9.98e-01 1.00e+00h  1
  12  1.1987501e+01 1.12e+00 9.64e+03  -1.0 1.86e+05    -  1.00e+00 1.00e+00h  1
  13  1.1987469e+01 6.54e-04 1.13e+01  -1.0 4.69e+03    -  1.00e+00 1.00e+00h  1
  14  1.1170398e+01 2.44e+02 2.58e+06  -5.7 3.79e+04    -  9.90e-01 7.32e-01f  1
  15  1.1170388e+01 6.94e-04 5.94e+01  -5.7 9.20e+00  -4.0 1.00e+00 1.00e+00h  1
  16  1.1170383e+01 1.01e+01 1.79e-01  -5.7 7.13e+00  -4.5 1.00e+00 1.00e+00h  1
  17  1.1170386e+01 1.13e-04 3.26e-02  -5.7 7.13e+00  -5.0 1.00e+00 1.00e+00h  1
  18  1.1170386e+01 8.82e-06 6.68e-06  -5.7 9.51e-03  -5.4 1.00e+00 1.00e+00h  1
  19  1.1170385e+01 1.19e-07 1.46e-05  -7.0 2.81e-02  -5.9 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.1170385e+01 1.79e-07

Upstream EF =  9.13
The initial DOF is 8
Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collec

   6  1.1358400e+01 7.22e+00 2.09e+04  -1.0 1.01e+04    -  2.39e-01 1.00e+00f  1
   7  1.1457340e+01 1.07e+00 3.50e+03  -1.0 1.30e+04    -  8.32e-01 1.00e+00f  1
   8  1.1761084e+01 4.84e+01 2.88e+03  -1.0 7.58e+04    -  9.22e-01 1.00e+00f  1
   9  1.1956967e+01 9.20e+02 2.76e+04  -1.0 8.25e+05    -  9.88e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.1952740e+01 5.45e+02 2.20e+05  -1.0 3.30e+06    -  9.90e-01 4.89e-01h  1
  11  1.1957727e+01 3.58e+01 2.48e+05  -1.0 1.04e+06    -  1.00e+00 1.00e+00h  1
  12  1.1959134e+01 9.71e+00 7.45e+04  -1.0 2.02e+05    -  1.00e+00 1.00e+00h  1
  13  1.1958874e+01 1.44e-01 1.80e+03  -1.0 4.57e+04    -  1.00e+00 1.00e+00h  1
  14  1.1139083e+01 2.98e+02 2.39e+06  -2.5 4.81e+04    -  1.00e+00 7.54e-01f  1
  15  1.1141987e+01 1.09e-03 9.79e+01  -2.5 1.68e+01  -4.0 1.00e+00 1.00e+00h  1
  16  1.1137424e+01 5.25e-06 7.80e-04  -3.8 9.97e-01  -4.5 1.00e+00 1.00e+00f  1
  17  1.1131743e+01 1.70e+02

====================  Region = EF-11  ======================
   region    butene    pentene     hexene    heptene     octene     nonene
0    EF-1  8.003895  16.720381  23.700189  25.553942  25.305838  18.616581
1    EF-2  7.993243  16.706787  23.673014  25.531558  25.327222  18.639088
2    EF-3  8.267204  16.111273  22.203526  23.208699  23.551002  17.298764
3    EF-4  8.267252  16.111359  22.203694  23.208816  23.550890  17.298630
4    EF-5  8.267312  16.111465  22.203900  23.208960  23.550752  17.298465
5    EF-6  7.651742  15.209476  22.120527  23.664824  25.850657  19.100533
6    EF-7  7.738653  15.188147  22.094733  23.475763  25.629285  18.923721
7    EF-8  7.999614  15.323760  22.168154  23.291697  25.266399  18.682949
8    EF-9  8.012938  14.700779  21.950876  22.530270  25.124226  18.436162
9   EF-10  8.165483  14.511869  21.884899  21.968059  24.859504  18.115029
10  EF-11  8.083054  13.795487  21.521665  21.014033  24.486729  17.580665

 Region = EF-12 

Upstream EF =  13.06